# Import

In [1]:
import numpy as np
import pandas as pd
import pandas.tseries.offsets as offsets
import datetime as dt
import time
import importlib
import logging
from retry import retry
from sqlalchemy import create_engine
from sqlalchemy.types import Date, Integer, Text

import stock

In [ ]:
importlib.reload(stock)

# 超大事。浅いコピーと深いコピー

## 浅いコピー。コピーというか参照なので変更内容は両方に反映される

In [ ]:
df = tables[11]

## 深いコピー。独立したコピーを作りたい場合は.copy()メソッドで

In [ ]:
df = tables[11].copy()

#

In [ ]:
# all_stock_tableにあってdomestic_stock_tableにない'code'を持つ行を抽出
all_stock_table[~all_stock_table['code'].isin(domestic_stock_table['code'])]

In [ ]:
# marketの種別で集計
all_stock_table.groupby('market').count()

In [ ]:
# 指定した値をNaNに置換、NaNはfloat型
all_stock_table.replace('-', np.NaN)

In [ ]:
 # 列単位で個別に名称を変更する場合
all_stock_table = all_stock_table.rename(columns={'市場・商品区分': 'market'})

In [ ]:
# marketの値を指定して選択
len(all_stock_table.query("market == '市場第一部（内国株）' | market == '市場第二部（内国株）'"))

In [29]:
# 'Date'列をインデックスに指定してCSVの読み込み、インデックスをdatetime型に変換
df_quote = pd.read_csv('/Users/Really/Stockyard/_csv/t_1758.csv', index_col='Date')
df_quote.index = pd.to_datetime(df_quote.index)
df_quote

,Open,High,Low,Close,Volume,AdjClose
Date,,,,,,
2000-01-04,575,575,575,575,1000,575
2000-01-06,565,575,565,565,3000,565
2000-01-11,565,570,565,565,5000,565
2000-01-12,565,570,565,570,2000,570
2000-01-17,570,570,570,570,2000,570
2000-01-19,580,585,580,580,4000,580
2000-01-20,585,585,585,585,1000,585
2000-01-21,575,575,575,575,1000,575
2000-01-25,575,575,575,575,1000,575


In [30]:
df_quote.dtypes

Open        int64
High        int64
Low         int64
Close       int64
Volume      int64
AdjClose    int64
dtype: object

In [31]:
df_quote.index

DatetimeIndex(['2000-01-04', '2000-01-06', '2000-01-11', '2000-01-12',
               '2000-01-17', '2000-01-19', '2000-01-20', '2000-01-21',
               '2000-01-25', '2000-01-26',
               ...
               '2017-10-10', '2017-10-17', '2017-10-18', '2017-10-19',
               '2017-10-20', '2017-10-23', '2017-10-25', '2017-10-30',
               '2017-10-31', '2017-11-01'],
              dtype='datetime64[ns]', name='Date', length=3166, freq=None)

In [27]:
# datetime型インデックスの作成例
dtidx = pd.date_range('2000-01-01', '2017-12-01', freq='B') # freq='B'はBusiness Day
dtidx

DatetimeIndex(['2000-01-03', '2000-01-04', '2000-01-05', '2000-01-06',
               '2000-01-07', '2000-01-10', '2000-01-11', '2000-01-12',
               '2000-01-13', '2000-01-14',
               ...
               '2017-11-20', '2017-11-21', '2017-11-22', '2017-11-23',
               '2017-11-24', '2017-11-27', '2017-11-28', '2017-11-29',
               '2017-11-30', '2017-12-01'],
              dtype='datetime64[ns]', length=4675, freq='B')

In [28]:
# 他のdatetime型インデックスにデータをあてはめる
quote_bd = pd.DataFrame(df_quote, index=pd.date_range(df_quote.index[0], df_quote.index[-1], freq='B'))
quote_bd

,Open,High,Low,Close,Volume,AdjClose
2000-01-04,575.0,575.0,575.0,575.0,1000.0,575.0
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,565.0,575.0,565.0,565.0,3000.0,565.0
2000-01-07,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-10,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-11,565.0,570.0,565.0,565.0,5000.0,565.0
2000-01-12,565.0,570.0,565.0,570.0,2000.0,570.0
2000-01-13,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-14,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-17,570.0,570.0,570.0,570.0,2000.0,570.0


In [ ]:
df['発表日'].apply(lambda x: parse(x, yearfirst=True).date())

In [ ]:
df['決算期'].apply(lambda x: datetime.strptime(x, '%Y.%m').date())

In [ ]:
tables[11].isnull()

In [ ]:
tables[11].isnull().any()

In [ ]:
# null が存在する行を取り除いて価格データとする 参考 https://qiita.com/u1and0/items/fd2780813b690a40c197
result = tmp_price[~tmp_price.isnull().any(axis=1)].astype(float).astype(int) # この場合、"~"は "== False" とするのと同じこと

In [ ]:
# 価格と価格以外の情報を分離
tmp_info = tmp_price[tmp_price.isnull().any(axis=1)].reset_index()

In [ ]:
# 全ての列項目がnullの行を排除
tables[11][~tables[11].isnull().all(axis=1)]

In [ ]:
# リスト内包表記。pandasではなくリストの話
csv_table = [i for i in csv_table if re.search(r't_\d*.csv', x)]

In [ ]:
# str アクセサ
[tables[11]['決算期'].str.contains('前期比')]

In [ ]:
# 複数条件による判定
# https://qiita.com/HirofumiYashima/items/fa76119d29bcb6e0bae7
# この場合、"|"ではなく"or"を使うとなぜかうまくいかない
tables[11][~((tables[11]['決算期'].str.contains('予')) | (tables[11]['決算期'].str.contains('前期比')))]

In [ ]:
# 行の要素を分割、元の列と新しく追加する列にそれぞれ代入
df[['会計基準', '決算期']] = pd.DataFrame(list(df['決算期'].str.split(' ')))

In [ ]:
# インデックスを振り直す
tables[11].reset_index(drop=True)

In [ ]:
# 列の並べ替えは列名のリストで
df = df[['会計基準', '決算期', '売上高', '営業益', '経常益', '最終益', '１株益', '１株配', '発表日']]

In [ ]:
# 列ごとに関数適用
# applyは遅いという話もあるので要検討。 http://sinhrks.hatenablog.com/entry/2015/07/11/223124
df[['売上高', '営業益', '経常益', '最終益']] = df[['売上高', '営業益', '経常益', '最終益']].apply(lambda x: x * 1000000)

# MySQLに接続

In [ ]:
sql = stock.sql()

In [ ]:
help(sql)

# MySQLに接続 (クラス不使用)

In [ ]:
db_settings = {
    "host": 'localhost',
    "database": 'StockPrice_Yahoo_1',
    "user": 'user',
    "password": 'password',
    "port":'3306'
}
engine = create_engine('mysql://{user}:{password}@{host}:{port}/{database}'.format(**db_settings))

# - Memo - MySQL クエリ

In [ ]:
# mysql>
CREATE DATABASE StockPrice_Yahoo_1 DEFAULT CHARACTER SET utf8mb4;
GRANT ALL ON StockPrice_Yahoo_1.* TO user@localhost IDENTIFIED BY 'password';
show databases;
use StockPrice_Yahoo_1;
show tables;
drop tables
drop database
select*from
show columns from # テーブルの中に含まれるカラムの情報を取得する
select*from t_1382 where Date between '2013-12-01' and '2013-12-31';
select * from t_8848 order by Date desc limit 5; # 最後の5行